**Load Librairies**

In [1]:
import asyncio
import gc
import os
import re
from typing import NoReturn

import librosa
import numpy as np
import pandas as pd
import pylab
import yt_dlp as yt

**Load Data To Download**

In [2]:
df = pd.read_csv("../../data/scraped/complete_dataset.csv", sep=",")[37224:]

**Download Audio Files & Automatic Conversion To CSV**

In [3]:
class YTDLSource:
    @classmethod
    async def from_url(cls, url, ytdl, *, loop=None):
        loop = loop or asyncio.get_event_loop()
        data = await loop.run_in_executor(
            None, lambda: ytdl.extract_info(url)
        )
        if "entries" in data:
            data = data["entries"][0]
        return ytdl.prepare_filename(data)

In [4]:
def save_spectrogram_image(spectrogram, output_file, y_axis):
    pylab.figure(figsize=(12, 8))
    pylab.axis('off')  # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])  # Remove the white edge
    librosa.display.specshow(spectrogram, y_axis=y_axis, x_axis='time', fmax=None, cmap='viridis')
    pylab.savefig(output_file, dpi=300, bbox_inches=None, pad_inches=0)
    pylab.close()

In [5]:
async def download_audio_files(row, loop) -> NoReturn:
    try:
        filename = f"{row[0]}_{row[1]}".replace(" ", "_")
        filename = re.sub('[^\w\s]', '', filename)
        path = f"../../data/musics/{filename}"
        audio_file = f"{path}.wav"
        ytdl_format_options = {
            "outtmpl": path,
            "format": "bestaudio/best",
            "postprocessors": [
                {
                    "key": "FFmpegExtractAudio",
                    "preferredcodec": "wav",
                    "preferredquality": "192",
                }
            ],
            "noplaylist": False,
            "nocheckcertificate": True,
            "ignoreerrors": False,
            "quiet": True,
            "no_warnings": True,
            "default_search": "auto",
        }
        ytdl = yt.YoutubeDL(ytdl_format_options)
        await asyncio.wait_for(YTDLSource.from_url(f"{row[0]}_{row[1]}", ytdl=ytdl, loop=loop), timeout=60)

        y, sr = librosa.load(audio_file)

        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        np.savetxt(f"../../data/dataset/raw/mfcc/{row[2]}/{filename}.csv", mfcc, delimiter=",")
        save_spectrogram_image(mfcc, f"../../data/dataset/img/mfcc/{row[2]}/{filename}.png", y_axis='linear')
        print(f"MFCCed : {audio_file}")

        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        np.savetxt(f"../../data/dataset/raw/chroma/{row[2]}/{filename}.csv", chroma_stft, delimiter=",")
        save_spectrogram_image(chroma_stft, f"../../data/dataset/img/chroma/{row[2]}/{filename}.png",
                               y_axis='chroma')
        print(f"Chromaed : {audio_file}")

        os.remove(audio_file)
        del y, sr, mfcc, chroma_stft
    except Exception:
        print(f"Error : {audio_file}")

In [6]:
async def start():
    for row in df.values:
        await download_audio_files(row, asyncio.get_event_loop())
        gc.collect()

In [7]:
await start()

              MFCCed : ../../data/musics/N_Sync__Gloria_Estefan_Music_Of_My_Heart.wav
Chromaed : ../../data/musics/N_Sync__Gloria_Estefan_Music_Of_My_Heart.wav
              MFCCed : ../../data/musics/10_years_novacaine.wav
Chromaed : ../../data/musics/10_years_novacaine.wav
              MFCCed : ../../data/musics/10_years_fix_me.wav
Chromaed : ../../data/musics/10_years_fix_me.wav
              MFCCed : ../../data/musics/10_years_shoot_it_out.wav
Chromaed : ../../data/musics/10_years_shoot_it_out.wav


ERROR: [youtube] 113_jackpot: Video unavailable


Error : ../../data/musics/113_jackpot.wav
              MFCCed : ../../data/musics/12_stones_we_are_one.wav
Chromaed : ../../data/musics/12_stones_we_are_one.wav
              MFCCed : ../../data/musics/2_In_A_Room_El_Trago_The_Drink.wav
Chromaed : ../../data/musics/2_In_A_Room_El_Trago_The_Drink.wav
              MFCCed : ../../data/musics/2_Of_Clubs_Walk_Tall.wav
Chromaed : ../../data/musics/2_Of_Clubs_Walk_Tall.wav
              MFCCed : ../../data/musics/3rd_bass_the_gas_face.wav
Chromaed : ../../data/musics/3rd_bass_the_gas_face.wav
              MFCCed : ../../data/musics/3rd_bass_the_cactus.wav
Chromaed : ../../data/musics/3rd_bass_the_cactus.wav
              MFCCed : ../../data/musics/3rd_bass_pop_goes_the_weasel.wav
Chromaed : ../../data/musics/3rd_bass_pop_goes_the_weasel.wav
              MFCCed : ../../data/musics/5_seconds_of_summer_ghost_of_you.wav
Chromaed : ../../data/musics/5_seconds_of_summer_ghost_of_you.wav
              MFCCed : ../../data/musics/50_cent_hit_you_u

ERROR: [youtube] Anne_Oliver: Video unavailable


Error : ../../data/musics/Anne_Oliver.wav
              MFCCed : ../../data/musics/Archie_Bell__The_Drells_Dancing_To_Your_Music.wav
Chromaed : ../../data/musics/Archie_Bell__The_Drells_Dancing_To_Your_Music.wav
              MFCCed : ../../data/musics/Aretha_Franklin_Willing_To_Forgive.wav
Chromaed : ../../data/musics/Aretha_Franklin_Willing_To_Forgive.wav
              MFCCed : ../../data/musics/Aretha_Franklin_with_The_Dixie_Flyers_Spirit_In_The_Dark.wav
Chromaed : ../../data/musics/Aretha_Franklin_with_The_Dixie_Flyers_Spirit_In_The_Dark.wav
              MFCCed : ../../data/musics/Ashton_Gardner__Dyke_Resurrection_Shuffle.wav
Chromaed : ../../data/musics/Ashton_Gardner__Dyke_Resurrection_Shuffle.wav
              MFCCed : ../../data/musics/Average_White_Band_Lets_Go_round_Again.wav
Chromaed : ../../data/musics/Average_White_Band_Lets_Go_round_Again.wav
              MFCCed : ../../data/musics/Aïoli_Les_vilains.wav
Chromaed : ../../data/musics/Aïoli_Les_vilains.wav
              MF

...

Error : ../../data/musics/hazhe_crazy.wav
              MFCCed : ../../data/musics/he_is_we_i_wouldnt_mind.wav
Chromaed : ../../data/musics/he_is_we_i_wouldnt_mind.wav
              MFCCed : ../../data/musics/heather_victoria_not_taking_you_back_feat_rapsody.wav
Chromaed : ../../data/musics/heather_victoria_not_taking_you_back_feat_rapsody.wav
              MFCCed : ../../data/musics/hellyeah_hush.wav
Chromaed : ../../data/musics/hellyeah_hush.wav
              MFCCed : ../../data/musics/hellyeah_love_falls.wav
Chromaed : ../../data/musics/hellyeah_love_falls.wav
              MFCCed : ../../data/musics/high_valley_dear_life.wav
Chromaed : ../../data/musics/high_valley_dear_life.wav
              MFCCed : ../../data/musics/high_valley_i_be_u_be.wav
Chromaed : ../../data/musics/high_valley_i_be_u_be.wav
              MFCCed : ../../data/musics/highly_suspect_bloodfeather.wav
Chromaed : ../../data/musics/highly_suspect_bloodfeather.wav
              MFCCed : ../../data/musics/highly_susp

ERROR: [youtube] jaden_ghost: Video unavailable


Error : ../../data/musics/jaden_ghost.wav
              MFCCed : ../../data/musics/jaden_watch_me.wav
Chromaed : ../../data/musics/jaden_watch_me.wav
            MFCCed : ../../data/musics/jagged_edge_where_the_party_at_rerecorded_remastered.wav
Chromaed : ../../data/musics/jagged_edge_where_the_party_at_rerecorded_remastered.wav
              MFCCed : ../../data/musics/jake_bugg_broken.wav
Chromaed : ../../data/musics/jake_bugg_broken.wav
              MFCCed : ../../data/musics/jake_bugg_two_fingers.wav
Chromaed : ../../data/musics/jake_bugg_two_fingers.wav
              MFCCed : ../../data/musics/jake_owen_señorita.wav
Chromaed : ../../data/musics/jake_owen_señorita.wav
              MFCCed : ../../data/musics/jake_owen_made_for_you.wav
Chromaed : ../../data/musics/jake_owen_made_for_you.wav
              MFCCed : ../../data/musics/jake_owen_i_was_jack_you_were_diane.wav
Chromaed : ../../data/musics/jake_owen_i_was_jack_you_were_diane.wav
              MFCCed : ../../data/musics/jak

ERROR: [youtube] khalid_talk: Video unavailable


Error : ../../data/musics/khalid_talk.wav
              MFCCed : ../../data/musics/khalid_this_way.wav
Chromaed : ../../data/musics/khalid_this_way.wav
              MFCCed : ../../data/musics/kid_bloom_different_state_of_mind.wav
Chromaed : ../../data/musics/kid_bloom_different_state_of_mind.wav
              MFCCed : ../../data/musics/kid_frost_la_raza.wav
Chromaed : ../../data/musics/kid_frost_la_raza.wav
              MFCCed : ../../data/musics/kid_rock_god_bless_saturday.wav
Chromaed : ../../data/musics/kid_rock_god_bless_saturday.wav
              MFCCed : ../../data/musics/kids_see_ghosts_4th_dimension.wav
Chromaed : ../../data/musics/kids_see_ghosts_4th_dimension.wav
              MFCCed : ../../data/musics/killah_priest_black_august_dark.wav
Chromaed : ../../data/musics/killah_priest_black_august_dark.wav
              MFCCed : ../../data/musics/killah_priest_musifixtion.wav
Chromaed : ../../data/musics/killah_priest_musifixtion.wav
              MFCCed : ../../data/musics/kil